#  OCR (Optical Character Recognition) - Experimento
## Utilização das bibliotecas [opencv](https://opencv.org/) e  [Tesseract OCR](https://tesseract-ocr.github.io/) para o reconhecimento de texto em imagens e da biblioteca [JiWER](https://github.com/jitsi/jiwer) para cálculo de mérticas de perfomance

*   Mais detlalhes sobre  o funcionamento dos algorítimos e das línguas nos quais o mesmo podem são utilizados são encontrados na [Tesseract documentation](https://tesseract-ocr.github.io/tessdoc/Data-Files)

*   Caso seja passado um arquivo .xlsx com as strings de target pode visualizar a perfonrmance do algorítimo



## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
O parâmetro `dataset` identifica os conjuntos de dados. Você pode importar arquivos de dataset com o botão  na barra de ferramentas.

In [4]:
dataset = "/tmp/data/OCR_dataset.zip" #@param {type:"string"}
target = "target_OCR" #@param {type:"string", label:"Atributo alvo", description:"Seu modelo tentará prever os valores do alvo."}
filter_type = "incluir" #@param ["incluir"] {type:"string",label:"Modo de seleção das features",description:"Neste caso está travdo em incluir porque o OCR precisa apenas do caminho da imagem"}
model_features = "input_image" #@param {type:"string",description:"Seu modelo será feito considerando apenas as features selecionadas. Caso nada seja especificado, todas as features serão utilizadas"}

#Hyperparams
bbox_conf = 60 #@param {type:"number",label:"Confiabilidade do bbox", description:"O quanto de confiabilidade o algorítmo deve possuir sobre o bbox para que o mesmo apareça."}

#Pytesseact Params
segmentation_mode = "Assume a single uniform block of text."  #@param ["Orientation and script detection (OSD) only.","Automatic page segmentation with OSD.","Automatic page segmentation, but no OSD, or OCR.","Fully automatic page segmentation, but no OSD. (Default)","Assume a single column of text of variable sizes.","Assume a single uniform block of vertically aligned text.","Assume a single uniform block of text.","Treat the image as a single text line.","Treat the image as a single word.","Treat the image as a single word in a circle.","Treat the image as a single character.","Sparse text. Find as much text as possible in no particular order.","Sparse text with OSD","Raw line. Treat the image as a single text line, bypassing hacks that are Tesseract-specific."] {type:"string",label:"Modeo de segmentação do PyTesseract",description:"Para mais informações acesse a documentação linkada no inicio do notebook"}
ocr_engine = "Neural nets LSTM engine only." #@param ["Legacy engine only.","Neural nets LSTM engine only.","Legacy + LSTM engines.","Default, based on what is available."] {type:"string",label:"OCR engine do Pytesseract",description:"Para mais informações acesse a documentação linkada no inicio do notebook"}
language = "por" #@param ["por","eng"] {type:"string",label:"Idioma pré teinado",description:"Para mais informações acesse a documentação linkada no inicio do notebook"}

#Return formart
bbox_return = "image" #@param ["np_array","image"] {type:"string",label:"Forma de retorno dos bboxes",description:"Escolher se bboxes serão retornados na imagem ou como um numpy array"}
image_return_format = ".jpg" #@param ["N/A",".jpg",".png"] {type:"string",label:"Formato de retorno da imagem caso bbox_return = image",description:"Escolher formato de retorno da imagem, N/A se retornar numpy array"}

## Acesso ao conjunto de dados

O conjunto de dados utilizado nesta etapa será o mesmo carregado através da plataforma.<br>
O tipo da variável retornada depende do arquivo de origem:
- [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) para CSV e compressed CSV: .csv .csv.zip .csv.gz .csv.bz2 .csv.xz
- [Binary IO stream](https://docs.python.org/3/library/io.html#binary-i-o) para outros tipos de arquivo: .jpg .wav .zip .h5 .parquet etc

In [5]:
import os
folder = os.path.join("/tmp/data","OCR_dataset")

!mkdir -p {folder}
!unzip -o {dataset} -d {folder}

Archive:  /tmp/data/OCR_dataset.zip
  inflating: /tmp/data/OCR_dataset/motiv3.jpg  
  inflating: /tmp/data/OCR_dataset/motiv2.jpg  
  inflating: /tmp/data/OCR_dataset/motiv1.png  
  inflating: /tmp/data/OCR_dataset/OCR_dataset.xlsx  


In [6]:
folder = os.path.join("/tmp/data","OCR_dataset")
dataset_path = ""
image_paths = []
for file in os.listdir(folder):
    if file.endswith(".xlsx"):
        dataset_path = os.path.join(folder, file)
    else:
        image_paths.append(os.path.join(folder, file))
dataset_path,image_paths

('/tmp/data/OCR_dataset/OCR_dataset.xlsx',
 ['/tmp/data/OCR_dataset/motiv2.jpg',
  '/tmp/data/OCR_dataset/motiv1.png',
  '/tmp/data/OCR_dataset/motiv3.jpg'])

In [7]:
import pandas as pd

df = pd.read_excel(dataset_path)
df["input_image"] = folder + "/" + df["input_image"]
df

,input_image,target_OCR
0,/tmp/data/OCR_dataset/motiv1.png,"Se não puder fazer tudo, faça tudo o que puder"
1,/tmp/data/OCR_dataset/motiv2.jpg,Comece acreditando que é possível
2,/tmp/data/OCR_dataset/motiv3.jpg,"Ainda que doa, valerá a pena. Esquece o medo v..."


## Acesso aos metadados do conjunto de dados

Utiliza a função `stat_dataset` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para carregar metadados.<br>
Por exemplo, arquivos CSV possuem `metadata['featuretypes']` para cada coluna no conjunto de dados (ex: categorical, numerical, or datetime).

In [9]:
import numpy as np

columns = df.columns.to_numpy()
target_index = np.argwhere(columns == target)
columns = np.delete(columns, target_index)
columns

array(['input_image'], dtype=object)

## Remoção de linhas com valores faltantes no atributo alvo
Caso haja linhas em que o atributo alvo contenha valores faltantes, é feita a remoção dos casos faltantes.


In [10]:
from sklearn.preprocessing import LabelEncoder

df.dropna(subset=[target], inplace=True)
df.dropna(subset=[columns[0]], inplace=True)
y = df[target].to_numpy()
y

array(['Se não puder fazer tudo, faça tudo o que puder',
       'Comece acreditando que é possível',
       'Ainda que doa, valerá a pena. Esquece o medo voa. Só voa.'],
      dtype=object)

## Filtragem das features


In [11]:
if filter_type == 'incluir':
    if len(model_features) >= 1:
        columns_index = (np.where(np.isin(columns,model_features)))[0]
        columns_index.sort()
        columns_to_filter = columns[columns_index]
        #featuretypes = featuretypes[columns_index]
    else:
        columns_to_filter = columns
else:
    if len(model_features) >= 1:
        columns_index = (np.where(np.isin(columns,model_features)))[0]
        columns_index.sort()
        columns_to_filter = np.delete(columns,columns_index)
        #featuretypes = np.delete(featuretypes,columns_index)
    else:
        columns_to_filter = columns

# keep the features selected
df = df[columns_to_filter]
X = df.to_numpy()
X

array([['/tmp/data/OCR_dataset/motiv1.png'],
       ['/tmp/data/OCR_dataset/motiv2.jpg'],
       ['/tmp/data/OCR_dataset/motiv3.jpg']], dtype=object)

## Chamada da Classe de OCR

In [12]:
hyperparams = {'bbox_conf':bbox_conf}
model_parameters = {'ocr_engine':ocr_engine,'segmentation_mode':segmentation_mode,'language':language}
return_formats = {'bbox_return':bbox_return,'image_return_format':image_return_format}

In [13]:
!wget https://raw.githubusercontent.com/platiagro/tasks/main/tasks/cv-ocr/ocr.py

--2020-10-26 11:40:02--  https://raw.githubusercontent.com/platiagro/tasks/main/tasks/cv-ocr/ocr.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.92.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.92.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7172 (7.0K) [text/plain]
Saving to: ‘ocr.py’

ocr.py              100%[===================>]   7.00K  --.-KB/s    in 0s      

2020-10-26 11:40:03 (26.8 MB/s) - ‘ocr.py’ saved [7172/7172]



In [14]:
from ocr import Class_Pytesseract_OCR
model = Class_Pytesseract_OCR(hyperparams,model_parameters)
model.get_result_dataframe(X,y)

,source_text,target_ocr_text,predicted_ocr_text,"BBOXES_COORDS(X, Y, W, H)",Match Error Rate (MER),Word Error Rate (WER),Word Information Lost (WIL),Word Information Preserved (WIP)
0,/tmp/data/OCR_dataset/motiv1.png,"Se não puder fazer tudo, faça tudo o que puder","Se não\npuder fazer\ntudo,\n\nfaça tudo\nque p...","[(126, 517, 147, 102), (304, 517, 238, 102), (...",0.300000,0.300000,0.510000,0.490000
1,/tmp/data/OCR_dataset/motiv2.jpg,Comece acreditando que é possível,COMECE\nACREDITANDO\nOJja\nPOSSÍVEL (9\nYV\n,"[(347, 680, 387, 75), (207, 826, 663, 75), (31...",0.714286,1.000000,0.885714,0.114286
2,/tmp/data/OCR_dataset/motiv3.jpg,"Ainda que doa, valerá a pena. Esquece o medo v...","Ainda que doa, valerá a\npena. Esquece o medo ...","[(138, 215, 71, 16), (225, 219, 41, 17), (282,...",0.142857,0.166667,0.142857,0.857143


## Salva métricas

Utiliza a função `save_metrics` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para salvar as métricas:`MER`, `WER`, `WIL`, `WIP` <br>


In [15]:
from platiagro import save_metrics

save_metrics(MER=model.avg_mer, WER=model.avg_wer, WIL=model.avg_wil, WIP=model.avg_wip)

## Salva modelo e outros resultados do treinamento

Escreve todos artefatos na pasta `/tmp/data/`. A plataforma guarda os artefatos desta pasta para usos futuros como implantação e comparação de resultados.

In [16]:
from joblib import dump

artifacts = {
    "hyperparams": hyperparams,
    "model_parameters": model_parameters,
    "return_formats":return_formats
}

dump(artifacts, "/tmp/data/ocr.joblib")

['/tmp/data/ocr.joblib']